# Orange brix Analysis

## Import thư viện

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from enum import Enum
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
import pickle

import os

In [15]:
plt.style.use("fivethirtyeight")

## Các cài đặt khác

In [16]:
# Biến GENERATE_PLOTS dùng để bật tắt việc vẽ đồ thị.
# Tắt đi để tăng tốc chạy test. (không chạy những hàm plot)
GENERATE_PLOTS: bool = False

# Biến boolean để bật tắt việc traing model khi ấn "Run all"
TRAIN_MODEL_1: bool = False
TRAIN_MODEL_2: bool = False
TRAIN_MODEL_3: bool = False
TRAIN_MODEL_4: bool = False
TRAIN_MODEL_5: bool = False

## Tạo các Class

### Class Utilities

In [17]:
# Class Utilities có các phương thức tiện ích.
# Các class khác muốn sử dụng các tiện ích này thì
# chỉ cần kế thừa class Utilities này.

class Utilities:
    def __init__(self):
        pass

    def get_categorical_numeric_cols(self, dataframe: pd.DataFrame):
        if len(dataframe) == 0:
            raise Exception("No records found!")

        numeric_cols = []
        categorical_cols = []

        for col in dataframe.columns:
            if isinstance(dataframe[col][0], bool) or not isinstance(
                dataframe[col][0], (int, float, np.int8, np.int16, np.int32, np.int64, np.float16, np.float32, np.float64)
            ):
                categorical_cols.append(col)
            else:
                numeric_cols.append(col)

        return (categorical_cols, numeric_cols)

### Enum ScalerType

In [18]:
# Khai báo Enum của các loại Scaler data từ submoule
# preprocessing của thư viện sklearn.

from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    RobustScaler,
    StandardScaler,
)


class ScalerType(Enum):
    Raw_ = None
    MaxAbsScaler_ = MaxAbsScaler
    MinMaxScaler_ = MinMaxScaler
    Normalizer_ = Normalizer
    RobustScaler_ = RobustScaler
    StandardScaler_ = StandardScaler

### Class Dataset

In [19]:
# Class Dataset chứa 1 dataframe và các metadata của
# dataframe này (tên, loại scaler, tên cột biến định
# danh, tên cột biến định lượng)


class Dataset(Utilities):
    def __init__(
        self,
        name: str,
        dataframe: pd.DataFrame,
        scaler_type: ScalerType = ScalerType.Raw_,
    ):
        self.name: str = name
        self.dataframe: pd.DataFrame = dataframe
        self.categorical_cols: list[str] = []
        self.numeric_cols: list[str] = []
        self.scaler_type: ScalerType = scaler_type

        self.categorical_cols, self.numeric_cols = self.get_categorical_numeric_cols(
            self.dataframe
        )

    def get_name(self) -> str:
        return self.name

    def get_dataframe(self) -> pd.DataFrame:
        return self.dataframe

    def detect_categorical_numeric_cols(self):
        if len(self.dataframe) == 0:
            raise Exception("No records found!")

        self.numeric_cols = []
        self.categorical_cols = []

        for col in self.dataframe.columns:
            try:
                float(self.dataframe[col][0])
                self.numeric_cols.append(col)
            except:
                self.categorical_cols.append(col)

    def get_numeric_cols(self) -> list[str]:
        return self.numeric_cols

    def get_numeric_dataframe(self) -> pd.DataFrame:
        return self.dataframe[self.numeric_cols]

    def get_categorical_dataframe(self) -> pd.DataFrame:
        return self.dataframe[self.categorical_cols]

    def get_scaler_type(self) -> ScalerType:
        return self.scaler_type

    def concat_dataframe(self, additional_dataframe: pd.DataFrame):
        if len(additional_dataframe) == len(self.dataframe):
            self.dataframe = pd.concat([self.dataframe, additional_dataframe], axis=1)
            self.detect_categorical_numeric_cols()

    def get_one_hot_vectorized_dataset(
        self, categorical_col: str, print_name: bool = True
    ):
        if categorical_col in self.categorical_cols:
            dummies_dataframe = pd.get_dummies(self.dataframe[categorical_col])
            dummies_dataframe = pd.concat([dummies_dataframe, self.dataframe], axis=1)
            new_dataset_name = self.name + "_ohv_" + categorical_col
            if print_name:
                print(new_dataset_name)
            return Dataset(
                name=new_dataset_name,
                dataframe=dummies_dataframe,
                scaler_type=self.scaler_type,
            )
        elif categorical_col in self.numeric_cols:
            print("Records for input column name are not categorical!")
        else:
            print("Input column name does not exists!")

### Enum ModelType

In [20]:
# ModelType là Enum khai báo các loại model từ thư viện
# sklearn và một số mô hình từ nguồn khác (xgb, lightgbm).
# Mỗi loại model được khai báo một bộ tham số đi kèm.


class ModelType(Enum):
    LINEAR_REGRESSION: dict = {}
    SVR: dict = {
        "kernel": "rbf",  # | linear, poly, rbf
        "degree": 3,
        "gamma": "scale",
        "coef0": 0,
        "tol": 1e-3,
        "C": 1.0,  # 0.1 - 2 step 0.1
        "epsilon": 0.1,
    }
    DECISION_TREE: dict = {
        "criterion": "squared_error",
        "splitter": "best",
        "max_depth": None,  # 1 - number of feature
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "min_weight_fraction_leaf": 0.0,
        "max_features": None,  # int 2->5 {“sqrt”, “log2”}, default=None
        "random_state": 42,
        "max_leaf_nodes": None,
        "min_impurity_decrease": 0,
    }
    RANDOM_FOREST: dict = {
        "n_estimators": 100,  # int 100-1000 (step 100)
        "criterion": "squared_error",
        "max_depth": None,  # 1 - number of feature
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "min_weight_fraction_leaf": 0.0,
        "max_features": 1.0,  # int 2->5 {“sqrt”, “log2”}, default=None
        "max_leaf_nodes": None,
        "min_impurity_decrease": 0.0,
    }
    LIGHT_GBM: dict = {
        "boosting_type": "gbdt",
        "num_leaves": 31,
        "max_depth": -1,
        "learning_rate": 0.1,
        "n_estimator": 100,  # int 100-1000 (step 100)
        "verbosity": -1,
    }
    XGB: dict = {
        "objective": "reg:linear",
        "n_estimators": 10,  # int 100-1000 (step 100)
        "seed": 123,
        "verbosity": 0,
    }

### Class Result

In [21]:
# Class Result dùng để chứa kết quả huấn luyện và các
# thông tin liên quan của một bộ (mô hình + dữ liệu).

# Một kết quả gồm:
# - Tên bộ dữ liệu
# - Tên của bộ tham số pretrained
# - Loại mô hình
# - Loại scaler
# - Kết quả r^2
# - Kết quả mse
# - Các tham số pretrained


class Result:
    def __init__(
        self,
        dataset_name: str,
        pretrained_model_features: str,
        model_type: ModelType,
        scaler_type: str,
        r2: float,
        mse: float,
        params: str,
    ):
        self.result = [
            dataset_name,
            pretrained_model_features,
            model_type,
            scaler_type,
            r2,
            mse,
            params,
        ]

    def get_result(self):
        return self.result

### Class ResultFrame

In [22]:
# Class ResultFrame chứa bảng chứa tất cả kết quả của
# một vòng huấn luyện mô hình.
# ResultFrame chứa một pd.DataFrame nên có thể hỗ trợ
# việc sắp xếp, lọc, xuất kết quả...


class ResultFrame:
    def __init__(self):
        self.result_frame: pd.DataFrame = pd.DataFrame()
        self.count = 0

        self.start_up()

    def start_up(self):
        self.result_frame["dataset_name"] = []
        self.result_frame["pretrained_model_features"] = []
        self.result_frame["model_type"] = []
        self.result_frame["scaler_type"] = []
        self.result_frame["r2"] = []
        self.result_frame["mse"] = []
        self.result_frame["params"] = []

    def add_result(self, new_result: Result):
        self.result_frame.loc[self.count] = new_result.result
        self.count += 1

    def display_result(self, display_rows: int = 30):
        self.result_frame.sort_values(["r2", "mse"], ascending=False, inplace=True)
        if display_rows < 0:
            display_rows = 30
        display(self.result_frame.head(display_rows))

    def save_result(self, filename=None, overwrite=False):
        if self.result_frame.shape[0] == 0:
            print("Result has no records!")
        elif not filename:
            filename = (
                self.result_frame["dataset_name"][0]
                + "_"
                + self.result_frame["pretrained_model_features"][0]
                + ".xlsx"
            )
            self.result_frame.to_excel(filename)
        else:
            file_exists = os.path.isfile(os.path.join(os.getcwd(), filename))
            if file_exists and not overwrite:
                print("Error: CANNOT SAVE FILE.")
                print("A file with the same name already exists.")
                print(
                    "Set overwrite to True to overwrite existed file or change file name!"
                )
            else:
                self.result_frame.to_excel(filename)

### Class Model (là class quan trọng nhất trong notebook này)

In [24]:
# Class Model dùng để tiền xử lý dữ liệu, huấn luyện và
# đánh giá mô hình với bộ dữ liệu đã cho ở đầu vào.

# Tham số đầu vào:
# - dataset_name: tên của Dataset
# - pretrained_model_features: tên của bộ tham số pretrained
# - model_type: loại Model
# - x_cols: một list chứa tên các cột sử dụng làm biến giải thích
# - y_cols: một list chứa tên các cột biến phản hồi
# - dataset: dataset mình muốn xử lý (dataset này có thể có
#       các cột không liên quan, các dữ liệu cho X và Y sẽ được
#       trích xuất tự động và các dữ liệu không liên quan sẽ bị loại bỏ)
# - scaler_type: loại scaler cho dữ liệu


class Model(Utilities):
    def __init__(
        self,
        dataset_name: str,
        pretrained_model_features: list[str],
        model_type: ModelType,
        x_cols: list[str],
        y_cols: list[str],
        dataset: Dataset,
        scaler_type: ScalerType = ScalerType.Raw_,
    ):

        if len(x_cols) == 0 or len(y_cols) == 0:
            raise Exception("Data columns are not specified!")

        self.dataset_name: str = dataset_name
        self.model_type: ModelType = model_type
        self.pretrained_model_features = str(pretrained_model_features)
        self.scaler_type: ScalerType = scaler_type
        self.scaler = self.get_scaler(self.scaler_type)  # could be None
        self.params = self.model_type.value
        self.x_dataset: Dataset = None
        self.y_dataset: Dataset = None
        self.inner_model = None
        self.r2 = None
        self.mse = None

        self.preprocess_data(x_cols, y_cols, dataset)

    # Extract data from x_cols and y_cols
    # Redetect categorical and numeric columns
    def preprocess_data(self, x_cols: list[str], y_cols: list[str], dataset: Dataset):
        x_dataset_new_name = dataset.get_name() + "_x"
        y_dataset_new_name = dataset.get_name() + "_y"
        scaler_type = dataset.get_scaler_type()
        self.x_dataset = Dataset(
            name=x_dataset_new_name,
            dataframe=dataset.get_dataframe()[x_cols],
            scaler_type=scaler_type,
        )
        self.y_dataset = Dataset(
            name=y_dataset_new_name,
            dataframe=dataset.get_dataframe()[y_cols],
            scaler_type=scaler_type,
        )

        self.process_x_dataset()

    # arrange [numeric_cols, categorical_cols]
    def process_x_dataset(self):
        new_name = self.x_dataset.get_name()
        numeric_dataframe = self.x_dataset.get_numeric_dataframe()
        categorical_dataframe = self.x_dataset.get_categorical_dataframe()
        scaler_type = self.x_dataset.get_scaler_type()
        self.x_dataset = Dataset(
            name=new_name,
            dataframe=pd.concat([numeric_dataframe, categorical_dataframe], axis=1),
            scaler_type=scaler_type,
        )

    def get_x_list_index_numeric_cols(self):
        return list(range(len(self.x_dataset.get_numeric_cols())))

    def fit_scaler_and_scale_dataframe(self, X: pd.DataFrame) -> pd.DataFrame:
        if self.scaler is not None:
            list_index = self.get_x_list_index_numeric_cols()
            X_numeric = X[:, list_index]
            X_categorical = X[:, len(list_index) :].astype(int)
            self.scaler.fit(X_numeric)
            transformed_X = self.scaler.transform(X_numeric)
            transformed_X = np.concatenate([transformed_X, X_categorical], axis=1)
        else:
            transformed_X = X

        return transformed_X

    def get_x_dataframe(self):
        return self.x_dataset.get_dataframe()

    def get_y_dataframe(self):
        return self.y_dataset.get_dataframe()

    def get_scaler(self, scaler_type: ScalerType):
        if scaler_type != ScalerType.Raw_:
            return scaler_type.value()

    def display_progress(self):
        print(
            f"Training | ModelType: {self.model_type} | Datasetname: {self.dataset_name} | ScalerType: {self.scaler_type} | Params: {self.params}"
        )
        print(f"R2: {self.r2} | MSE: {self.mse}")

    def fit(self, X=None, Y=None):
        if X is None:
            X = np.array(self.x_dataset.get_dataframe())
            Y = np.array(self.y_dataset.get_dataframe())

        model_family = self.model_type.__str__().split(".")[-1]

        Y = np.squeeze(Y)

        # need to scale
        transformed_X = self.fit_scaler_and_scale_dataframe(X)

        if model_family == "SVR":
            self.model = SVR(**(self.params)).fit(transformed_X, Y)
        elif model_family == "LINEAR_REGRESSION":
            self.model = LinearRegression(**(self.params)).fit(transformed_X, Y)
        elif model_family == "DECISION_TREE":
            self.model = DecisionTreeRegressor(**(self.params)).fit(transformed_X, Y)
        elif model_family == "RANDOM_FOREST":
            self.model = RandomForestRegressor(**(self.params)).fit(transformed_X, Y)
        elif model_family == "LIGHT_GBM":
            self.model = LGBMRegressor(**(self.params)).fit(transformed_X, Y)
        elif model_family == "XGB":
            self.model = XGBRegressor(**(self.params)).fit(transformed_X, Y)
        else:
            raise ValueError("Unknown model family!")

    def predict(self, X):
        if self.scaler is not None:
            list_index = self.get_x_list_index_numeric_cols()
            X_numeric = X[:, list_index]
            X_categorical = X[:, len(list_index) :].astype(int)
            transformed_X = self.scaler.transform(X_numeric)
            transformed_X = np.concatenate([transformed_X, X_categorical], axis=1)
        else:
            transformed_X = X
        return self.model.predict(transformed_X)

    def evaluate(self, X=None, Y=None, method="LOOCV"):
        if X is None:
            X = np.array(self.x_dataset.get_dataframe())
            Y = np.array(self.y_dataset.get_dataframe())
        if method == "LOOCV":
            preds = [0] * len(Y)
            total = len(Y)
            kf = KFold(n_splits=total)
            kf.get_n_splits(Y)

        with tqdm(total=total) as pbar:
            for i, (train_index, valid_index) in enumerate(kf.split(X, Y)):
                X_train = X[train_index]
                Y_train = Y[train_index]
                X_valid = X[valid_index]
                Y_valid = Y[valid_index]

                self.fit(X_train, Y_train)

                Y_valid_pred = self.predict(X_valid)
                for j in range(len(valid_index)):
                    index = valid_index[j]
                    value = Y_valid_pred[j]
                    preds[index] = value
                pbar.update(1)

        self.r2 = r2_score(Y, preds)
        self.mse = mean_squared_error(Y, preds)

        self.display_progress()

        # Compile result
        return Result(
            dataset_name=self.dataset_name,
            pretrained_model_features=self.pretrained_model_features,
            model_type=self.model_type.name,
            scaler_type=self.scaler_type.name,
            r2=self.r2,
            mse=self.mse,
            params=self.params,
        )

    def save(self, filename: str):
        pickle.dump(self.inner_model, open(filename, "wb"))

    def load(self, filename: str):
        pickle.dump(self.inner_model, open(filename, "wb"))

    def set_params(self, params: dict):
        self.params = params

### Các biến global trong notebook